In [3]:
import os
from pathlib import Path
from weasyprint import HTML

# Leer archivo CURATE para obtener: cita, command line y datos CURATE
def format_data(file_path, start_str, end_str, every=100):
    """Reads a file and returns a formatted string between two markers."""
    with open(file_path, 'r', errors="replace") as file:
        content = file.read()
        start_pos = content.find(start_str) - 3
        end_pos = content.find(end_str, start_pos)
        end_pos = content.find("\n", end_pos)
        data = content[start_pos:end_pos]
        lines = data.split('\n')
        formatted_lines = [
            insert_newlines('    ' + line[4:] if line.startswith('- ') else line, every)
            for line in lines
        ]
        return '\n'.join(formatted_lines)

def insert_newlines(string, every=100):
    """Inserts a newline character every `every` characters in the string."""
    return '\n'.join(string[i:i+every] for i in range(0, len(string), every))

modules = ['CURATE','GENERATE','VERIFY','PREDICT']
starting_word = ["Starting","Starting","Starting tests","Representation"]
for i,module in enumerate(modules):
    format_data(Path(f'{module}_data.dat'), starting_word[i], f"Time {module}:")

with open(Path('../CURATE_data.dat'), 'r', errors="replace") as datfile:
    first_two_lines = datfile.readline() + datfile.readline() + datfile.readline()
    last_line = datfile.readline()
    formatted_last_line = insert_newlines(last_line.strip(), 114)
    first_two_lines += formatted_last_line

predict_dir = Path('../PREDICT')
verify_dir = Path('../VERIFY')
dir_depth = 2

# Find matching files in PREDICT directory
predict_files = [str(file_path) for file_path in predict_dir.rglob('*PFI*.png') if file_path.is_file()]

# Generate HTML code for PREDICT images
html_predict_png = ''.join([f'<img src="{image_path}" alt="" class="img-predict"/>' for image_path in predict_files])

# Find matching files in VERIFY directory
verify_files = [str(file_path) for file_path in verify_dir.rglob('*PFI*.png') if file_path.is_file()]

# Generate HTML code for VERIFY images
html_verify_png = ''.join([f'<img src="{image_path}" alt="" class="img-predict"/>' for image_path in verify_files])

# Generar los HTML
first_two_lines_html = '<div class="dat-content"><pre>' + first_two_lines + '</pre></div>'   
curate_html = '<div class="dat-content"><pre>' + curated_data + '</pre></div>'
predict_html = '<div class="predict-content"><pre>' + predict_data + '</pre></div>'
verify_html = '<div class="verify-content"><pre>' + verify_data + '</pre></div>'
generate_dat_html = '<div class="dat-content"><pre>' + generated_data + '</pre></div>'

# Generar el HTML del REPORT completo

report_html = """
<h1 style="text-align: left; margin-bottom: 0.5em;">
    Report
    <img src="report/Robert_logo.jpg" alt="" style="float:right; width:25%;" />
    <br>
    <hr class="black">
    <span style="font-weight:bold;"></span>
</h1>
<p style="text-align: center;"></p>
<p style="text-align: justify;"
{}
<hr>
<p><span style="font-weight:bold;">
    <img src="report/content-curation.png" alt="" style="width:20px; height:20px; margin-right:5px;">
    CURATE 
</span></p>
{}
<img src="../CURATE/Pearson_heatmap.png" alt="" />
<p>
<hr>
<p><span style="font-weight:bold;">
    <img src="report/cpu.png" alt="" style="width:20px; height:20px; margin-right:5px;">
    GENERATE 
</span></p>
{}
<img src="../GENERATE/Raw_data/Heatmap ML models no PFI filter.png" alt="" />
<img src="../GENERATE/Raw_data/Heatmap ML models with PFI filter.png" alt="" />
<p>
<hr>
<p><span style="font-weight:bold;">
    <img src="report/analysis.png" alt="" style="width:20px; height:20px; margin-right:5px;">
    PREDICT 
</span></p>
{}
{}
<p>
<hr>
<p><span style="font-weight:bold;">
    <img src="report/verified.png" alt="" style="width:20px; height:20px; margin-right:5px;">
    VERIFY 
</span></p>
{}
{}
<p>
<hr>
<p style="text-align: justify;"><span style="font-weight:bold;">AQME-ROBERT</span></p>
<img src="report/AQME_ROBERT.png" alt="" />
</span>
""".format(first_two_lines_html, curate_html,generate_dat_html,predict_html,html_predict_png,verify_html,html_verify_png)

# Guardar el HTML generado en un archivo
with open("report.html", "w") as htmlfile:
    htmlfile.write(report_html)
    
# Generar archivo CSS

css_content = css_content()
def css_content():
    css_content = """
    body {
    font-size: 12px;
    }
    @page :first {
        @top-left {
            content: "";
        }
        @top-right {
            content: "";
        }
        @bottom-left {
            content: "ROBERT v 0.0.1";
            font-size: 8pt;
            position: fixed;
            left: 0;
            bottom: 0;
        }
        @bottom-right {
            content: counter(page) " of " counter(pages);
            font-size: 8pt;
            position: fixed;
            right: 0;
            bottom: 0;
        }
    }
    @page {
        size: A4;
        margin: 2cm;
        @top-left {
            content: "ROBERT report";
            font-size: 8pt;
            position: fixed;
            left: 0;
            top: 0;
        }
        @top-right {
            content: "CSV file name";
            font-size: 8pt;
            position: fixed;
            left: 0;
            top: 0;
        }
        @bottom-left {
            content: "ROBERT v 0.0.1";
            font-size: 8pt;
            position: fixed;
            left: 0;
            bottom: 0;
        }
        @bottom-right {
            content: counter(page) " of " counter(pages);
            font-size: 8pt;
            position: fixed;
            right: 0;
            bottom: 0;
        }
    }
    * {
        font-family: Arial, sans-serif;
    }
    .dat-content {
        width: 50%;
        max-width: 595pt;
        overflow-x: auto;
        line-height: 1.2;
    }

    img[src="Robert_logo.jpg"] {
        float: right;
        width: 50%;
    }
    img[src*="Pearson"] {
        display: inline-block;
        vertical-align: bottom;
        max-width: 48%;
        margin-left: 10px;
        margin-bottom: -5px;
    }

    img[src*="PFI"] {
        display: inline-block;
        vertical-align: bottom;
        max-width: 48%;
        margin-left: 10px;
        margin-bottom: -5px;
    }

    img[src*="PFI"]:first-child {
        margin-right: 10%;
    }
    .img-predict {
        margin-top: 20px;
    }
    hr.black {
    border: none;
    height: 1px;
    background-color: black;
    }

    hr {
    border: none;
    height: 1px;
    background-color: gray;
    }

    body:before {
    top: 1.2cm;
    }
    """
    return css_content
# Escribir el contenido del archivo CSS en un archivo
with open("report.css", "w") as cssfile:
    cssfile.write(css_content)
    
#Creación archivo pdf
def makepdf(html, css_files=None):
    """Generate a PDF file from a string of HTML."""
    htmldoc = HTML(string=html, base_url="")
    if css_files:
        htmldoc = htmldoc.write_pdf(stylesheets=css_files)
    else:
        htmldoc = htmldoc.write_pdf()
    return htmldoc


def run():
    """Command runner."""
    html_file = "report.html"  # Ruta del archivo HTML
    css_files = ["report.css"]  # Ruta del archivo CSS
    outfile = "Report.pdf"  # Ruta del archivo PDF de salida
    html = Path(html_file).read_text()
    pdf = makepdf(html, css_files=css_files)
    Path(outfile).write_bytes(pdf)


if __name__ == "__main__":
    run()
